In [1]:
import trainer
from utils import batchify
import json
import tensorflow as tf
import numpy as np

In [2]:
with open('SentimentAnalysis/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('SentimentAnalysis/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)

In [3]:
VERSION = '102_sa'
params = dict(
    model_configs = {
      "rnn_layers":[
          {
             "units": 1024,
             "drop_i": 0.05,
             "wdrop": 0.1,
             "drop_o": 0.05
          },
          {
             "units": 1024,
             "wdrop": 0.1,
             "drop_o": 0.05
          },
          {
             "units": 1024,
             "drop_o": 0.05,
             "wdrop": 0.1
          }
],
       "vocab_size": len(word2idx) + 1,
       "drop_e": 0.0,
       "char_vocab_size": len(char2idx) + 1,
       "char_cnn_options": {
           "layers": [
                [1, 16],
                [2, 16],
                [3, 32],
                [4, 64],
                [5, 128],
                [6, 256],
                [7, 512]
           ],
           "n_highways": 2
       },
       "char_vec_size": 16,
       "projection_dims": 512,
       "skip_connection": True
    },
    optimizer = tf.train.GradientDescentOptimizer,
    negative_samples = 10240,
    wdecay = 1.2e-6,
    alpha = 1e-6,
    beta = 1e-6,
    clip_norm = 0.3,
    bptt = 70,
    use_ema = True,
    save_freq = 1000,
    log_path = '{}/logs'.format(VERSION),
    train_summary_dir = '{}/train_summary/'.format(VERSION),
    test_summary_dir = '{}/test_summary/'.format(VERSION),
    checkpoint_dir = '{}/checkpoints/'.format(VERSION)
)

# In[12]:


my_trainer = trainer.Trainer(**params, fine_tune=True)

my_trainer.logger.info('Trainer params {}'.format(params))
# In[13]:


tf.reset_default_graph()
my_trainer.build()

Trainer params {'model_configs': {'rnn_layers': [{'units': 1024, 'drop_i': 0.05, 'wdrop': 0.1, 'drop_o': 0.05}, {'units': 1024, 'wdrop': 0.1, 'drop_o': 0.05}, {'units': 1024, 'drop_o': 0.05, 'wdrop': 0.1}], 'vocab_size': 153531, 'drop_e': 0.0, 'char_vocab_size': 1005, 'char_cnn_options': {'layers': [[1, 16], [2, 16], [3, 32], [4, 64], [5, 128], [6, 256], [7, 512]], 'n_highways': 2}, 'char_vec_size': 16, 'projection_dims': 512, 'skip_connection': True}, 'optimizer': <class 'tensorflow.python.training.gradient_descent.GradientDescentOptimizer'>, 'negative_samples': 10240, 'wdecay': 1.2e-06, 'alpha': 1e-06, 'beta': 1e-06, 'clip_norm': 0.3, 'bptt': 70, 'use_ema': True, 'save_freq': 1000, 'log_path': '102_sa/logs', 'train_summary_dir': '102_sa/train_summary/', 'test_summary_dir': '102_sa/test_summary/', 'checkpoint_dir': '102_sa/checkpoints/'}


INFO:tensorflow:Restoring parameters from 102_sa/checkpoints/train/model.cpkt-1046000


Restoring parameters from 102_sa/checkpoints/train/model.cpkt-1046000


In [4]:
with open('SentimentAnalysis/train_word.npy','rb') as inp:
    train_word = np.load(inp)
with open('SentimentAnalysis/train_char.npy','rb') as inp:
    train_char = np.load(inp)
with open('SentimentAnalysis/test_word.npy','rb') as inp:
    valid_word = np.load(inp)
with open('SentimentAnalysis/test_char.npy','rb') as inp:
    valid_char = np.load(inp)

In [5]:
train_word = batchify(train_word, 25).T
train_char = batchify(train_char, 25).T
valid_word = batchify(valid_word, 11).T
valid_char = batchify(valid_char, 11).T

In [6]:
# Reset global step to zero
my_trainer.session.run(tf.assign(my_trainer.global_step, 0))

0

In [7]:
start_i = 0
lr = 10.0
fine_tune_rate = [1/3, 1/3, 1/3, 18]
for _ in range(5):
    my_trainer.train_dev_loop(train_word, train_char, valid_word, valid_char, lr, start_i=start_i, folder_test='test_sa', folder_train='train_sa', fine_tune_rate=fine_tune_rate)
    lr = max(lr / np.sqrt(2.0), 2.0)
    start_i = 0

Step    1: progress 69/7364,forward loss 5.86737, backward loss 5.85811, ppl 351.69, bpc 08.46, time 12.22
Step    2: progress 133/7364,forward loss 5.57976, backward loss 5.45562, ppl 249.06, bpc 07.96, time 12.84
Step    3: progress 194/7364,forward loss 5.38652, backward loss 5.52704, ppl 234.34, bpc 07.87, time 13.36
Step    4: progress 229/7364,forward loss 5.49692, backward loss 5.62396, ppl 259.94, bpc 08.02, time 13.80
Step    5: progress 302/7364,forward loss 6.06486, backward loss 5.83070, ppl 382.90, bpc 08.58, time 14.45
Step    6: progress 377/7364,forward loss 5.98474, backward loss 5.62868, ppl 332.52, bpc 08.38, time 15.07
Step    7: progress 446/7364,forward loss 5.76153, backward loss 5.50022, ppl 278.91, bpc 08.12, time 15.72
Step    8: progress 467/7364,forward loss 5.16548, backward loss 5.16354, ppl 174.95, bpc 07.45, time 16.06
Step    9: progress 542/7364,forward loss 5.29544, backward loss 4.98833, ppl 171.04, bpc 07.42, time 16.77
Step   10: progress 603/7364,

Step   77: progress 5083/7364,forward loss 4.56878, backward loss 4.54828, ppl 95.44, bpc 06.58, time 51.30
Step   78: progress 5155/7364,forward loss 4.55572, backward loss 4.62491, ppl 98.53, bpc 06.62, time 51.75
Step   79: progress 5227/7364,forward loss 4.70823, backward loss 4.72607, ppl 111.85, bpc 06.81, time 52.18
Step   80: progress 5295/7364,forward loss 4.84023, backward loss 4.52250, ppl 107.92, bpc 06.75, time 52.62
Step   81: progress 5364/7364,forward loss 4.50907, backward loss 4.57346, ppl 93.81, bpc 06.55, time 53.06
Step   82: progress 5437/7364,forward loss 4.43154, backward loss 4.59047, ppl 91.01, bpc 06.51, time 53.49
Step   83: progress 5506/7364,forward loss 4.96642, backward loss 4.72345, ppl 127.10, bpc 06.99, time 53.93
Step   84: progress 5571/7364,forward loss 4.67125, backward loss 4.74649, ppl 110.93, bpc 06.79, time 54.34
Step   85: progress 5643/7364,forward loss 4.62035, backward loss 4.68074, ppl 104.64, bpc 06.71, time 54.80
Step   86: progress 571

Step  129: progress 1298/7364,forward loss 4.38536, backward loss 4.08080, ppl 68.93, bpc 06.11, time 08.60
Step  130: progress 1364/7364,forward loss 4.41936, backward loss 4.21716, ppl 75.06, bpc 06.23, time 09.01
Step  131: progress 1432/7364,forward loss 4.37012, backward loss 4.49514, ppl 84.15, bpc 06.39, time 09.54
Step  132: progress 1496/7364,forward loss 4.18451, backward loss 4.20782, ppl 66.43, bpc 06.05, time 09.94
Step  133: progress 1565/7364,forward loss 4.19209, backward loss 4.15073, ppl 64.81, bpc 06.02, time 10.38
Step  134: progress 1598/7364,forward loss 4.42811, backward loss 4.23376, ppl 76.02, bpc 06.25, time 10.74
Step  135: progress 1661/7364,forward loss 4.50911, backward loss 4.29781, ppl 81.73, bpc 06.35, time 11.15
Step  136: progress 1720/7364,forward loss 4.39784, backward loss 4.28407, ppl 76.78, bpc 06.26, time 11.68
Step  137: progress 1789/7364,forward loss 4.51943, backward loss 4.31885, ppl 83.02, bpc 06.38, time 12.11
Step  138: progress 1862/736

Step  205: progress 6411/7364,forward loss 3.82180, backward loss 4.35406, ppl 59.62, bpc 05.90, time 43.01
Step  206: progress 6474/7364,forward loss 3.99098, backward loss 4.14539, ppl 58.45, bpc 05.87, time 43.53
Step  207: progress 6552/7364,forward loss 4.22137, backward loss 4.58952, ppl 81.90, bpc 06.36, time 44.13
Step  208: progress 6625/7364,forward loss 4.12636, backward loss 4.18283, ppl 63.73, bpc 05.99, time 44.58
Step  209: progress 6695/7364,forward loss 4.03299, backward loss 4.24951, ppl 62.88, bpc 05.97, time 45.03
Step  210: progress 6769/7364,forward loss 4.06260, backward loss 4.07592, ppl 58.51, bpc 05.87, time 45.58
Step  211: progress 6837/7364,forward loss 4.09111, backward loss 4.15569, ppl 61.77, bpc 05.95, time 45.99
Step  212: progress 6906/7364,forward loss 4.06963, backward loss 4.36298, ppl 67.78, bpc 06.08, time 46.46
Step  213: progress 6971/7364,forward loss 4.29830, backward loss 4.28986, ppl 73.26, bpc 06.20, time 46.90
Step  214: progress 7042/736

Step  257: progress 2562/7364,forward loss 3.73117, backward loss 4.10290, ppl 50.25, bpc 05.65, time 17.50
Step  258: progress 2629/7364,forward loss 3.90645, backward loss 3.93603, ppl 50.46, bpc 05.66, time 17.93
Step  259: progress 2701/7364,forward loss 3.84573, backward loss 4.05263, ppl 51.89, bpc 05.70, time 18.40
Step  260: progress 2760/7364,forward loss 4.00383, backward loss 4.06517, ppl 56.51, bpc 05.82, time 18.79
Step  261: progress 2805/7364,forward loss 3.84415, backward loss 4.00948, ppl 50.75, bpc 05.67, time 19.12
Step  262: progress 2875/7364,forward loss 4.07647, backward loss 3.85442, ppl 52.74, bpc 05.72, time 19.63
Step  263: progress 2950/7364,forward loss 3.91027, backward loss 3.89511, ppl 49.54, bpc 05.63, time 20.09
Step  264: progress 3021/7364,forward loss 3.89767, backward loss 3.77877, ppl 46.44, bpc 05.54, time 20.51
Step  265: progress 3091/7364,forward loss 3.74628, backward loss 3.85278, ppl 44.68, bpc 05.48, time 20.94
Step  266: progress 3150/736

Evaluate loss 4.582454681396484, time 3.7346408367156982
Evaluate loss 4.5438337326049805, time 3.981487274169922
Evaluate loss 4.638407230377197, time 4.2352213859558105
Evaluate loss 4.509677886962891, time 4.490297317504883
Evaluate loss 4.443968296051025, time 4.746062755584717
Evaluate loss 4.648690700531006, time 4.995110750198364
Evaluate loss 4.5127997398376465, time 5.2515904903411865
Evaluate loss 4.3984527587890625, time 5.513736963272095
Evaluate loss 4.623899459838867, time 5.771337985992432
Evaluate loss 4.647974014282227, time 6.0228025913238525
Evaluate loss 4.634796142578125, time 6.274431467056274
Evaluate loss 4.4022955894470215, time 6.521148920059204
Evaluate loss 4.496626377105713, time 6.7718141078948975
Evaluate loss 4.385594844818115, time 7.025719881057739
Evaluate loss 4.827673435211182, time 7.276433944702148
Evaluate loss 4.270887851715088, time 7.510023832321167
Evaluate loss 4.559372425079346, time 7.7647013664245605
Evaluate loss 4.379687309265137, time 

Step  386: progress 3825/7364,forward loss 3.87082, backward loss 3.74196, ppl 44.99, bpc 05.49, time 25.70
Step  387: progress 3855/7364,forward loss 3.63858, backward loss 3.64048, ppl 38.07, bpc 05.25, time 26.14
Step  388: progress 3933/7364,forward loss 3.65859, backward loss 3.65435, ppl 38.72, bpc 05.28, time 26.63
Step  389: progress 4000/7364,forward loss 3.64483, backward loss 3.69699, ppl 39.29, bpc 05.30, time 27.05
Step  390: progress 4066/7364,forward loss 3.75810, backward loss 3.50168, ppl 37.71, bpc 05.24, time 27.48
Step  391: progress 4136/7364,forward loss 3.86444, backward loss 3.60271, ppl 41.83, bpc 05.39, time 27.93
Step  392: progress 4212/7364,forward loss 3.67446, backward loss 3.69946, ppl 39.92, bpc 05.32, time 28.40
Step  393: progress 4277/7364,forward loss 3.64757, backward loss 3.72144, ppl 39.83, bpc 05.32, time 28.81
Step  394: progress 4347/7364,forward loss 3.61510, backward loss 3.56497, ppl 36.24, bpc 05.18, time 29.28
Step  395: progress 4416/736

Evaluate loss 4.407375812530518, time 13.551070928573608
Evaluate loss 4.530974388122559, time 13.804996013641357
Evaluate loss 4.576247215270996, time 13.974353313446045
Evaluate total loss 4.542190596909379, time 13.97494888305664
Step  440: progress 71/7364,forward loss 3.87567, backward loss 3.91308, ppl 49.13, bpc 05.62, time 00.46
Step  441: progress 139/7364,forward loss 3.74021, backward loss 3.82998, ppl 44.04, bpc 05.46, time 00.91
Step  442: progress 205/7364,forward loss 3.66161, backward loss 3.90291, ppl 43.92, bpc 05.46, time 01.35
Step  443: progress 276/7364,forward loss 3.82813, backward loss 3.62486, ppl 41.53, bpc 05.38, time 01.84
Step  444: progress 349/7364,forward loss 3.73902, backward loss 3.85306, ppl 44.52, bpc 05.48, time 02.29
Step  445: progress 421/7364,forward loss 3.87327, backward loss 3.76652, ppl 45.60, bpc 05.51, time 02.75
Step  446: progress 491/7364,forward loss 3.64614, backward loss 3.69167, ppl 39.21, bpc 05.29, time 03.21
Step  447: progress

Step  514: progress 5073/7364,forward loss 3.54404, backward loss 3.52154, ppl 34.22, bpc 05.10, time 34.00
Step  515: progress 5145/7364,forward loss 3.54712, backward loss 3.52392, ppl 34.31, bpc 05.10, time 34.46
Step  516: progress 5212/7364,forward loss 3.51180, backward loss 3.49425, ppl 33.22, bpc 05.05, time 34.89
Step  517: progress 5293/7364,forward loss 3.50526, backward loss 3.45270, ppl 32.43, bpc 05.02, time 35.41
Step  518: progress 5356/7364,forward loss 3.46015, backward loss 3.52868, ppl 32.93, bpc 05.04, time 35.85
Step  519: progress 5428/7364,forward loss 3.43113, backward loss 3.68710, ppl 35.13, bpc 05.13, time 36.35
Step  520: progress 5491/7364,forward loss 3.57381, backward loss 3.71672, ppl 38.29, bpc 05.26, time 36.76
Step  521: progress 5556/7364,forward loss 3.44493, backward loss 3.74204, ppl 36.36, bpc 05.18, time 37.20
Step  522: progress 5622/7364,forward loss 3.53495, backward loss 3.72958, ppl 37.80, bpc 05.24, time 37.61
Step  523: progress 5692/736